# 3C454.3 analysis notebook
Reference paper: https://arxiv.org/pdf/1106.5162.pdf

In [ ]:
from agilepy.api.AGAnalysis import AGAnalysis

In [ ]:
# Interactive plots
%matplotlib widget

## Creating a configuration file

In [ ]:
confFilePath = "$HOME/agilepy_conf.yaml"

In [ ]:
AGAnalysis.getConfiguration(
    confFilePath = confFilePath,
    evtfile="$AGILE/agilepy-test-data/test_dataset_agn/EVT/EVT.index",
    logfile="$AGILE/agilepy-test-data/test_dataset_agn/LOG/LOG.index",
    userName = "username",
    sourceName = "3C454_3",
    tmin = 55513.0,
    tmax = 55521.0,
    timetype = "MJD",
    glon = 86.11,
    glat = -38.18,
    outputDir = "$HOME/agilepy_analysis",
    verboselvl = 1
)

## Obtaining the AGAnalysis object

In [ ]:
ag = AGAnalysis(confFilePath)

In [ ]:
#print all options of the configuration file
ag.printOptions()

## Sources hypothesis

In [ ]:
sources = ag.loadSourcesFromCatalog("2AGL", rangeDist = (0, 5), show=True)

## Adding a source at runtime

In [ ]:
#newSourceDict = {
#    "glon" : 79.3,
#    "glat": 0.554,
#    "spectrumType" : "PowerLaw",
#    "flux": 1.69737e-07,
#    "index": 2.1
#}

#s = ag.addSource("CYGX3", newSourceDict)

#print(s)

## Deleting sources
Selection params = [name, dist, flux, sqrtts]

In [ ]:
deletedSources = ag.deleteSources('flux <= 10e-08', show = True)

## Selecting sources

In [ ]:
sources = ag.selectSources("flux > 0", show = True)

## Free a source's parameter
Freeable params = [flux, index, index1, index2, cutoffEnergy, pivotEnergy, curvature, pos]

In [ ]:
affectedSources = ag.freeSources('name == "2AGLJ2254+1609"', "flux", True, show=True)

In this example, only one source is affected.

## Analysis

In [ ]:
#Generate maps
maplistfile = ag.generateMaps()

In [ ]:
! cat $maplistfile

In [ ]:
#Display counts map
ag.displayCtsSkyMaps(maplistFile=maplistfile, smooth=3, catalogRegions="2AGL", catalogRegionsColor="green")

In [ ]:
#Display exposure map
ag.displayExpSkyMaps()

In [ ]:
#Display diffuse emission map
ag.displayGasSkyMaps()

In [ ]:
#Fix the diffuse emission coefficient to a default value. 
ag.setOptions(galcoeff=[0.7])

In [ ]:
! cat $maplistfile

In [ ]:
ag.printOptions("model")

In [ ]:
#calculate a mean value of isotropic and diffuse emission coefficients. 
gal,iso,maplist = ag.calcBkg("2AGLJ2254+1609", galcoeff = [0.7], pastTimeWindow = 0)

In [ ]:
print(iso)

In [ ]:
! cat $maplistfile

In [ ]:
ag.printOptions("model")

In [ ]:
#Perform a maximum likelihood estimator
ag.mle()

In [ ]:
#Display the results
ag.selectSources("sqrtTS > 0", show=True)

### Light curve with default tmin and tmax

In [ ]:
lightCurveData = ag.lightCurveMLE("2AGLJ2254+1609", binsize=86400)

In [ ]:
cat $lightCurveData

In [ ]:
print(lightCurveData)

In [ ]:
ag.displayLightCurve("mle")

### Light curve with explicit tmin and tmax

In [ ]:
lightCurveData = ag.lightCurveMLE("2AGLJ2254+1609", tmin=55513.0, tmax=55515.0, timetype="MJD", binsize=86400)

In [ ]:
cat $lightCurveData

In [ ]:
ag.displayLightCurve("mle")

### Light curve with default tmin and tmax and binsize of 43200 s

In [ ]:
lightCurveData = ag.lightCurveMLE("2AGLJ2254+1609", binsize=43200)

In [ ]:
ag.displayLightCurve("mle")

### Evaluation of spectral index of pre-flare period 55513.00-55517

In [ ]:
#the current version of the setOptions works only with TT 
from agilepy.utils.AstroUtils import AstroUtils
ag.setOptions(tmin = AstroUtils.time_mjd_to_tt(55513.0), tmax=AstroUtils.time_mjd_to_tt(55517.0), timetype="TT")
ag.setOptions(energybins=[[100, 300], [300, 1000], [1000, 3000], [3000, 10000]])

In [ ]:
ag.printOptions("maps")

In [ ]:
maplistfile = ag.generateMaps()

In [ ]:
affectedSources = ag.freeSources('name == "2AGLJ2254+1609"', "index", True, show=True)

In [ ]:
ag.mle()

In [ ]:
selectedSources = ag.selectSources('flux > 0')
print(len(selectedSources))
for s in selectedSources:
    print(s)

In [ ]:
lightCurveData = ag.lightCurveMLE("2AGLJ2254+1609", tmin=55513.0, tmax=55517.0, timetype="MJD", binsize=86400)

In [ ]:
ag.displayLightCurve("mle")

### Evaluation of spectral index of flare period 55517-55521

## Cleaning up

In [ ]:
ag.deleteAnalysisDir()